This program is used to acertain if a a piece of news is fake or not.

It does that by considering 

In [ ]:
import numpy as np 
import pandas as pd

import re #(re -regular expressions--used for seraching words in a text or paragraph)
from nltk.corpus import stopwords #(nltk-natural language toolkit)
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer #(Tfidf- used to change words to numbers)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [ ]:
import nltk
nltk.download('stopwords') #downloads stop words for various languages 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#printing the stopwords in english 
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [ ]:
#CSV means Comma Separated Value

In [ ]:
news_dataset = pd.read_csv('/content/sample_data/train.csv')

In [ ]:
news_dataset.shape

(20800, 5)

In [ ]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
#to replace the empty columns with null values
# '' fills with null n this case 
news_dataset = news_dataset.fillna('')

In [ ]:
# to get if its fake or not, we combine the title and authur
#not the text, because the text is very large 
news_dataset['content'] = news_dataset.author +' '+news_dataset.title

#                            #OR 

# news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset.title

In [ ]:
print(news_dataset.content)

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:
#separating our datasets 
#iloc doesnt start indexing at 0, it starts from one 
X = news_dataset.drop('label',axis=1)
Y = news_dataset.label

In [ ]:
news_dataset.head()


,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [ ]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [ ]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


The Stemming Procedure:
stemming is the process of reducing a word to its root words 

Example:actor,actress,acting--->act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower() #coverts our results from the first line to lower case
  stemmed_content = stemmed_content.split() # the words willthen be splitted and coverted into lists e.h abcde.split() = ['a','b','c','d']
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


# stemmed_content = re.sub('[^a-zA-Z]',' ',content)


# 1.(^) means exclussion,that tells us thatwe want a-zA-Z excluded from our removals
# 1.so if its not an alphabet, it is simply removed from the content table
# 1.if there are numbers or commas or quotes or stuff like that,it willbe replaced by quotes
# 1. middle quotes from the first line

In [ ]:
news_dataset['content'] =news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset.content)

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
X = news_dataset.content.values #addding values to this turns it into an array
Y = news_dataset.label.values 

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [ ]:
print(Y)

[1 0 1 ... 0 1 1]


In [ ]:
print(X.shape,Y.shape)

(20800,) (20800,)


In [ ]:
# now as you can see, X is still in word form,not in numbers,we cant feed words into our algo
# so we have to convert it to numbers that the ML algo can read..that is where 
# TfidfVectorizer comes in

In [ ]:
#converting texts to numerical data 

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
# TfidfVectorizer -term frequency inverse document frequency vectorizer 
# TF counts the number of occurences of a significant word in a text file and gives is a numerical Value to show itd importance in the file 
# IDF checks for the number of occurences of INsignificant word and gives a particular value to it (the insignificant word can occur a lot, depending on the file)

#  X = vectorizer.fit_transform(X), this is the same as what was done up, makes you understand better where everything had to be returned to X

In [ ]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Splitting to training and test data

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2 )
#stratify is used to segregate the Y data into equal proportions of 1's an 0's,
#it is used to split data in the same proportion that is present in the original dataset

training the model 

In [ ]:
model= LogisticRegression()


In [ ]:
model.fit(X_train,Y_train)

LogisticRegression()

Evalauation


In [ ]:
X_train_prediction = model.predict(X_train)

In [ ]:
training_prediction_accuracy = metrics.accuracy_score(Y_train,X_train_prediction)

In [ ]:
#the mathematics behind logistics regtression is in the video

In [ ]:
print('the training prediction accuracy is:', training_prediction_accuracy)

the training prediction accuracy is: 0.9865985576923076


In [ ]:
X_test_prediction = model.predict(X_test)

In [ ]:
test_training_accuracy = metrics.accuracy_score(Y_test,X_test_prediction)

In [ ]:
print('the test data accuracy is :', test_training_accuracy)

the test data accuracy is : 0.9790865384615385


Making a Predictive system 


In [ ]:
X_news = X_test[3]

prediction =model .predict(X_news)
print(prediction)

if prediction[0] ==0:
  print('the news is real')
else:
  print('the news is fake')

[0]
the news is real


In [ ]:
print(X_test)

  (0, 12801)	0.2910746804557067
  (0, 9818)	0.30786004182651133
  (0, 7668)	0.22945314906455008
  (0, 6816)	0.16094563145945953
  (0, 6289)	0.288254092437116
  (0, 5941)	0.288254092437116
  (0, 5233)	0.21316265672448448
  (0, 4346)	0.3250084367199054
  (0, 3395)	0.3301936745912874
  (0, 2959)	0.24534646237198773
  (0, 1667)	0.30373060380734146
  (0, 908)	0.213510750423647
  (0, 239)	0.34297808354766485
  (1, 16996)	0.09117761343372983
  (1, 15295)	0.08946281236254729
  (1, 14046)	0.42524648908354634
  (1, 13190)	0.36773046084789346
  (1, 12741)	0.24868518461414146
  (1, 12279)	0.3796661151115819
  (1, 12041)	0.37327055071909065
  (1, 10306)	0.08813410128297053
  (1, 8813)	0.42524648908354634
  (1, 4008)	0.23098933893199997
  (1, 3339)	0.2834482751186189
  (2, 16868)	0.344315415802567
  :	:
  (4158, 16782)	0.2796201559111399
  (4158, 13954)	0.2713157796360236
  (4158, 13918)	0.29432702492466434
  (4158, 13775)	0.21479309786895925
  (4158, 13600)	0.2496550139728973
  (4158, 11068)	0.2943

In [ ]:
print(Y_test[0])

1


In [ ]:
print(Y_test[3])

0
